# Topic Modelling with Term Frequency - Inverse Document Frequency

## Topic Predictor Exercise

Import libraries and initialize NLP collections for German; import a small archive of Kafka's short stories.

In [1]:
import nltk
import re
import spacy
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from kafka_stories import title_list, all_stories

stopwords_german = stopwords.words('german')
nlp = spacy.load('de_core_news_md')

Preprocess texts (clean, lemmatize, delete stopwords); store them as a list of strings (one string per story).

In [2]:
def clean_text(text):
    return re.sub(r"\W+", " ", text).lower()

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = ""
    for token in doc:
         if token.lemma_ != " " and token.lemma_ not in stopwords_german:
                lemmatized_text += str(token.lemma_) + " "
    return lemmatized_text

def preprocess_text(text):
    return lemmatize_text(clean_text(text))

preprocessed_stories = [preprocess_text(story) for story in all_stories]

print(all_stories[6])
print(preprocessed_stories[6])


Mein Großvater pflegte zu sagen: »Das Leben ist erstaunlich kurz. Jetzt
in der Erinnerung drängt es sich mir so zusammen, daß ich zum Beispiel
kaum begreife, wie ein junger Mensch sich entschließen kann ins nächste
Dorf zu reiten, ohne zu fürchten, daß – von unglücklichen Zufällen ganz
abgesehen – schon die Zeit des gewöhnlichen, glücklich ablaufenden
Lebens für einen solchen Ritt bei weitem nicht hinreicht.«

großvater pflegen sagen leben erstaunlich kurz erinnerung drängen zusammen beispiel kaum begreifen jung mensch entschließen nächst dorf reiten fürchten unglücklich zufällen ganz absehen schon zeit gewöhnlich glücklich ablaufend lebens solch ritt weit hinreichen 


Using CountVectorizer, create a vector for all the words in the archive and count their frequency in each story. Extract the list of features in the archive (words). Create a table for word frequencies using Panda. 

In [3]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(preprocessed_stories)

feature_names = vectorizer.get_feature_names()

df_word_counts = pd.DataFrame(counts.T.todense(), index=feature_names, columns=title_list)
print(df_word_counts)

             Der neue Advokat  Auf der Galerie  Ein altes Blatt  \
aas                         0                0                0   
ab                          0                0                0   
abend                       0                0                0   
abends                      0                0                0   
abfertigen                  0                0                0   
...                       ...              ...              ...   
übersehen                   0                0                0   
übersiedelt                 0                0                0   
überwinden                  0                0                0   
übrigens                    0                0                0   
übungen                     0                0                1   

             Vor dem Gesetz  Schakale und Araber  Ein Besuch im Bergwerk  \
aas                       0                    1                       0   
ab                        0                

Use TfidfTransformer to turn the Bag-of-Words frequency obtained from CountVectorizer into a Term Frequency - Inverse Document Frequency score. Use Pandas to create a table containing the tf-idf scores.

In [4]:
transformer = TfidfTransformer(norm=None)
tfidf_scores_transformed = transformer.fit_transform(counts)

df_tf_idf = pd.DataFrame(tfidf_scores_transformed.T.todense(), index=feature_names, columns=title_list)
print(df_tf_idf)

             Der neue Advokat  Auf der Galerie  Ein altes Blatt  \
aas                       0.0              0.0         0.000000   
ab                        0.0              0.0         0.000000   
abend                     0.0              0.0         0.000000   
abends                    0.0              0.0         0.000000   
abfertigen                0.0              0.0         0.000000   
...                       ...              ...              ...   
übersehen                 0.0              0.0         0.000000   
übersiedelt               0.0              0.0         0.000000   
überwinden                0.0              0.0         0.000000   
übrigens                  0.0              0.0         0.000000   
übungen                   0.0              0.0         2.704748   

             Vor dem Gesetz  Schakale und Araber  Ein Besuch im Bergwerk  \
aas                     0.0             2.704748                     0.0   
ab                      0.0             0.0

Finally, obtain the highest tf-idf score for each story and display it. Does it reflect the topic of the story? 

In [5]:
for story_title in title_list:
    print(df_tf_idf[[story_title]].idxmax())

Der neue Advokat    bucephalus
dtype: object
Auf der Galerie    pferde
dtype: object
Ein altes Blatt    nomaden
dtype: object
Vor dem Gesetz    türhüter
dtype: object
Schakale und Araber    araber
dtype: object
Ein Besuch im Bergwerk    herren
dtype: object
Das nächste Dorf    ablaufend
dtype: object
Eine kaiserliche Botschaft    botschaft
dtype: object
Die Sorge des Hausvaters    odradek
dtype: object
Ein Brudermord    schmar
dtype: object
